# Import

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
import numpy as np
import time

import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("..")

from flow_simulations.flow_sim import compute_solution

# Neural Network

In [ ]:
def create_model():
    model = keras.Sequential()

    for i in range(8):
        model.add(keras.layers.Dense(100,
                                     name='layer%d_' % i))
        model.add(tf.keras.layers.ReLU(max_value=1.0))
        
    # pv, sw, so, uw, uo
    model.add(keras.layers.Dense(5,
                                 activation=None,
                                 name='layer_out', ))
              
    return model

In [ ]:
def network(model, x, t, alpha_wat=0., alpha_oil=0., training=False):
    y = tf.stack([x, t], axis=-1)
    return model(y, training)

In [ ]:
model = create_model()

In [ ]:
model.summary()

# Visualize

In [ ]:
x_beg, x_end = 0, 1
t_beg, t_end = 0, 1

In [ ]:
N = 128
grids_xt = np.meshgrid(np.linspace(x_beg, x_end, N), np.linspace(t_beg, t_end, 33), indexing='ij')
grid_x, grid_t = [tf.convert_to_tensor(t, tf.float32) for t in grids_xt]

# create 4D tensor with batch and channel dimensions in addition to space and time
# in this case gives shape=(1, N, 33, 1)
# grid_u = tf.expand_dims(network(model, grid_x, grid_t), axis=0)
# grid_u.shape

In [ ]:
def show_state(a, title):
    for i in range(4): 
        a = np.concatenate([a, a], axis=3)
    
    a = np.reshape(a, [a.shape[1], a.shape[2] * a.shape[3]])
    
    fig, axes = plt.subplots(1, 1, figsize=(16, 5))
    im = axes.imshow(a, origin='upper', cmap='inferno')
    
    plt.colorbar(im);
    plt.xlabel('time');
    plt.ylabel('x');
    plt.title(title)
    plt.show()

# Loss

## Boundary loss

# boundary condition

In [ ]:
def zeros(shape):
    result = list(map(lambda val: 1 if val is None else val, shape))
    return np.array(np.zeros(result, np.int8)).astype(np.float32)

def ones(shape):
    result = list(map(lambda val: 1 if val is None else val, shape))
    return np.array(np.ones(result, np.int8)).astype(np.float32)

$$s_w(x>0, t = 0) = 0$$
$$s_{oil} = 1 - s_w$$
$$p(x, t = 0) = 1 - x$$

In [ ]:
def open_boundary_t(N):
    x = np.random.uniform(x_beg + 1e-10, x_end, [N]).astype(np.float32)
    t = np.asarray(np.zeros_like(x))
    
    pv = np.zeros_like(x)
    sw = np.zeros_like(x)
    so = np.zeros_like(x)
    uw = np.zeros_like(x)
    uo = np.zeros_like(x)
    for idx in range(N):
        _, _, _, _, uw[idx], uo[idx] = compute_solution(t[idx], x[idx], nx=100)
    
    pv = 1 - x.copy()
    sw = zeros([N])
    so = 1 - sw.copy()
    
    return x, t, pv, sw, so, uw, uo

In [ ]:
x, t, pv, sw, so, uw, uo = open_boundary_t(10)
x, t, sw, so, pv+x, uw, uo

$$p\left(x=0, t\in[0;1]\right)$$
$$s_w\left(x=0, t\in[0;1]\right)=1$$
$$p\left(x=1, t\in[0;1]\right)$$
$$s_{oil} = 1 - s_w$$

In [ ]:
def open_boundary_x(N):
    x = np.concatenate([zeros([N // 2]), zeros([N // 2]) + 1], axis=0)
    t = np.random.uniform(t_beg, t_end, [N]).astype(np.float32)
    
    pv = np.zeros_like(x)
    sw = np.zeros_like(x)
    so = np.zeros_like(x)
    uw = np.zeros_like(x)
    uo = np.zeros_like(x)
    for idx in range(N):
        pv[idx], _, _, so[idx], uw[idx], uo[idx] = compute_solution(t[idx], x[idx], nx=100)
    
    sw[:N//2] = ones([N//2])
    so = 1 - sw.copy()
    
    return x, t, pv, sw, so, uw, uo

In [ ]:
x, t, pv, sw, so, uw, uo = open_boundary_x(10)
x, t, pv, sw, so, uw, uo

In [ ]:
def boundary_tx(N):
    x = np.linspace(x_beg, x_end, 128)
    t = np.asarray(np.ones_like(x)) * 0.5
    
    pv = np.zeros_like(x)
    sw = np.zeros_like(x)
    so = np.zeros_like(x)
    uw = np.zeros_like(x)
    uo = np.zeros_like(x)
    for idx in range(N):
        pv[idx], _, sw[idx], so[idx], uw[idx], uo[idx] = compute_solution(t[idx], x[idx], nx=100)
    
    perm = np.random.permutation(128)
    return (x[perm])[0:N], (t[perm])[0:N], (pv[perm])[0:N], \
            (sw[perm])[0:N], (so[perm])[0:N], (uw[perm])[0:N], (uo[perm])[0:N]

def _ALT_t0(N):  # alternative, impose original initial state at t=0
    x = rnd.random_uniform(-1, 1, N)
    t = rnd.zeros_like(x)
    u = - math.sin(np.pi * x)
    return x, t, u

In [ ]:
def div(numerator, denominator):
    return tf.cast(numerator, np.float32) / tf.cast(denominator, np.float32)

def l_n_loss(tensor, n, batch_norm=True):
    total_loss = tf.reduce_sum(tensor ** n) / n
    if batch_norm:
        batch_size = tf.shape(tensor)[0]
        return div(total_loss, batch_size)
    else:
        return total_loss

def l2_loss(tensor):
    return l_n_loss(tensor, 2, batch_norm=True)   # normalizes by first dimension, N_bc

In [ ]:
N_SAMPLE_POINTS_BND = 100

x_bc, t_bc, pv_bc, sw_bc, so_bc, uw_bc, uo_bc = [np.concatenate([v_t0, v_x0, v_x], axis=0)
                    for v_t0, v_x0, v_x in zip(open_boundary_t(N_SAMPLE_POINTS_BND),
                                               open_boundary_x(N_SAMPLE_POINTS_BND),
                                               boundary_tx(N_SAMPLE_POINTS_BND))]

x_bc, t_bc, pv_bc, sw_bc, so_bc, uw_bc, uo_bc = \
                    (np.asarray(x_bc, dtype=np.float32),
                     np.asarray(t_bc, dtype=np.float32),
                     np.asarray(pv_bc, dtype=np.float32),
                     np.asarray(sw_bc, dtype=np.float32),
                     np.asarray(so_bc, dtype=np.float32),
                     np.asarray(uw_bc, dtype=np.float32),
                     np.asarray(uo_bc, dtype=np.float32))

res_bc = np.stack([pv_bc, sw_bc, so_bc, uw_bc, uo_bc], axis=1)

# with app.model_scope():
loss_init_bc = l2_loss(network(model, x_bc, t_bc) - res_bc)
loss_init_bc

## Physics loss inside of domain

$s_t, s_x, p_x, u_x$

$$f = (\phi \cdot s_t + u_x)^2 + \left(u + \frac{k(s)p_x}{\mu \cdot }\right)^2$$

$$loss = f_w + f_{oil}$$

$\alpha\in [1..6]$ - случайным образом

In [ ]:
# pv, sw, so, uw, uo
def k_wat(s, alpha_wat=2.0, k=1.0):
    eps = 1.0e-10
    sp = (s + eps) / (1.0 + eps)
    return k * (sp ** alpha_wat)

def k_oil(s, alpha_oil=4.0, k=0.1):
    eps = 1.0e-10
    sp = (s + eps) / (1.0 + eps)
    return k * (sp ** alpha_oil)

def f_water(model, x, t, training=False):
    """ Physics-based loss function"""
    with tf.GradientTape() as tape:
        s = network(model, x, t, training)[:, 1]
    s_t = tf.cast(tape.gradient(s, t), np.float32)    

    with tf.GradientTape() as t1:
        u = network(model, x, t, training)[:, 3]
    u_x = tf.cast(t1.gradient(u, x), np.float32)
    
    with tf.GradientTape() as t1:
        p = network(model, x, t, training)[:, 0]
    p_x = tf.cast(t1.gradient(p, x), np.float32)
    
    phi = 0.1
    mu = 1
        
    return (phi * s_t + u_x)**2 + (u + k_wat(s) * p_x / mu)**2

def f_oil(model, x, t, training=False):
    """ Physics-based loss function"""
    with tf.GradientTape() as tape:
        s = network(model, x, t, training)[:, 2]
    s_t = tf.cast(tape.gradient(s, t), np.float32)    

    with tf.GradientTape() as t1:
        u = network(model, x, t, training)[:, 4]
    u_x = tf.cast(t1.gradient(u, x), np.float32)
    
    with tf.GradientTape() as t1:
        p = network(model, x, t, training)[:, 0]
    p_x = tf.cast(t1.gradient(p, x), np.float32)
    
    phi = 0.1
    mu = 3
        
    return (phi * s_t + u_x)**2 + (u + k_oil(s)* p_x / (mu))**2

In [ ]:
# Physics loss inside of domain
N_SAMPLE_POINTS_INNER = 1000
x_ph, t_ph = (tf.Variable(np.random.uniform(x_beg, x_end, [N_SAMPLE_POINTS_INNER])),
              tf.Variable(np.random.uniform(t_beg, t_end, [N_SAMPLE_POINTS_INNER])))

loss_init_ph = l2_loss(f_water(model, x_ph, t_ph) + f_oil(model, x_ph, t_ph))  # normalizes by first dimension, N_ph
loss_init_ph

## Result Loss

In [ ]:
# Combine
ph_factor = 1. # get_loss_factor(loss_init_ph)
loss_value = loss_init_bc + ph_factor * loss_init_ph  # allows us to control the relative influence of loss_ph
loss_value, ph_factor

# Fit

In [ ]:
lr = 0.02
optimizer = keras.optimizers.SGD(learning_rate=lr)

In [ ]:
# ph_factor recalculate
start = time.time()
ITERS = 1000000
INNER_ITTERS = 10000

history_LU = []
history_LPh = []
history_L2 = []

grd_ph_factor = 1000

for epochs in range(ITERS+1):
    x_bc, t_bc, pv_bc, sw_bc, so_bc, uw_bc, uo_bc = [np.concatenate([v_t0, v_x0, v_x], axis=0)
                    for v_t0, v_x0, v_x in zip(open_boundary_t(N_SAMPLE_POINTS_BND),
                                               open_boundary_x(N_SAMPLE_POINTS_BND),
                                               boundary_tx(N_SAMPLE_POINTS_BND))]
    x_bc, t_bc, pv_bc, sw_bc, so_bc, uw_bc, uo_bc = \
                        (np.asarray(x_bc, dtype=np.float32),
                         np.asarray(t_bc, dtype=np.float32),
                         np.asarray(pv_bc, dtype=np.float32),
                         np.asarray(sw_bc, dtype=np.float32),
                         np.asarray(so_bc, dtype=np.float32),
                         np.asarray(uw_bc, dtype=np.float32),
                         np.asarray(uo_bc, dtype=np.float32))
    res_bc = np.stack([pv_bc, sw_bc, so_bc, uw_bc, uo_bc], axis=1)
    
    loss_init_bc = l2_loss(network(model, x_bc, t_bc) - res_bc)

    x_ph, t_ph = (tf.Variable(np.random.uniform(x_beg, x_end, [N_SAMPLE_POINTS_INNER])),
              tf.Variable(np.random.uniform(t_beg, t_end, [N_SAMPLE_POINTS_INNER])))

    loss_init_ph = l2_loss(f_water(model, x_ph, t_ph) + f_oil(model, x_ph, t_ph))  # normalizes by first dimension, N_ph

    print(f"loss_init_bc: {loss_init_bc}; loss_init_ph: {loss_init_ph}")

    if loss_init_bc < 1e-3: 
        grd_ph_factor /= 10.
        lr /= 10.
        optimizer = keras.optimizers.SGD(learning_rate=lr)
        print('learning_rate:%f; grd_ph_factor:%d ' % (lr, grd_ph_factor))
    
    for optim_step in range(INNER_ITTERS+1):
        with tf.GradientTape() as tape1:
            val_bc = network(model, x_bc, t_bc, False)
            loss_bc = tf.math.log(l2_loss(val_bc - res_bc) + loss_init_bc) - tf.math.log(loss_init_bc)
            history_LU.append(loss_bc)
            
            val_ph = f_water(model, x_ph, t_ph, False) + f_oil(model, x_ph, t_ph, False)
            loss_ph = l2_loss(val_ph)
            history_LPh.append(loss_ph)
            
            loss_bc_grad_free = np.log(l2_loss(val_bc - res_bc) + loss_init_bc) - np.log(loss_init_bc)
            ph_factor = 1 / (1+loss_bc_grad_free * grd_ph_factor)
            loss_value = loss_bc + ph_factor * loss_ph
            history_L2.append(loss_value)
        
        grads = tape1.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if optim_step > 0 and optim_step % 100 == 0 and loss_ph < 1:
            x_ph, t_ph = (tf.Variable(np.random.uniform(x_beg, x_end, [N_SAMPLE_POINTS_INNER])),
                          tf.Variable(np.random.uniform(t_beg, t_end, [N_SAMPLE_POINTS_INNER])))
    
            loss_init_ph = l2_loss(f_water(model, x_ph, t_ph) + f_oil(model, x_ph, t_ph))  # normalizes by first dimension, N_ph
            print('Step %d, loss: %f = loss_bc:%f + loss_ph:%f * ph_factor:%f' % (optim_step, loss_value, loss_bc, loss_ph, ph_factor))
            print(f"loss_init_bc: {loss_init_bc}; loss_init_ph: {loss_init_ph}")
        
        if optim_step<3 or optim_step % 1000 == 0:
            print('Step %d, loss: %f = loss_bc:%f + loss_ph:%f * ph_factor:%f; grd_ph_factor:%f' % (optim_step, loss_value, loss_bc, loss_ph, ph_factor, grd_ph_factor))
            model.save_weights(f"C:/PBDL/HC_grad_{grd_ph_factor}_lr_{str(lr).replace('.', '_')}/HC_{optim_step}")

        if np.abs(loss_value) < 1e-4:
            print('Step %d , loss: %f ' % (optim_step,loss_value))
            break

# end = time.time()
# print("Runtime {:.2f} s".format(end-start))